# Continuous Improvement Experiment
## Mini-Fulfillment Conveyor System

This notebook analyzes baseline vs. improved simulation runs to demonstrate
data-driven continuous improvement on the conveyor system.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from pathlib import Path
import glob
import numpy as np

# Plot styling
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 11

## 1. Load Data

Load the most recent baseline and improved run CSV files.

In [ ]:
# Find the most recent metrics CSVs in each folder
baseline_dir = Path('../baseline')
improved_dir = Path('../improved')

def load_latest_metrics(directory):
    """Load the most recent metrics CSV from a directory."""
    files = sorted(directory.glob('metrics_*.csv'))
    if not files:
        print(f'No metrics files found in {directory}')
        return None
    latest = files[-1]
    print(f'Loading: {latest}')
    df = pd.read_csv(latest)
    return df

def load_latest_events(directory):
    """Load the most recent events CSV from a directory."""
    files = sorted(directory.glob('events_*.csv'))
    if not files:
        print(f'No events files found in {directory}')
        return None
    latest = files[-1]
    print(f'Loading: {latest}')
    df = pd.read_csv(latest)
    return df

# Load data
df_baseline = load_latest_metrics(baseline_dir)
df_improved = load_latest_metrics(improved_dir)
ev_baseline = load_latest_events(baseline_dir)
ev_improved = load_latest_events(improved_dir)

## 2. Experiment Design

### Hypothesis

> **If we increase the jam detection timeout from 4.0s to 5.5s and add a
> minimum inter-arrival spacing check (300mm gap enforcement), we will
> reduce false jam detections and overall downtime without significantly
> impacting throughput.**

### Parameters Changed

| Parameter | Baseline | Improved |
|---|---|---|
| Jam timeout (rJamTimeoutSec) | 4.0 s | 5.5 s |
| Box arrival rate | 72/hr | 72/hr (unchanged) |
| Conveyor speed | 1.0 | 1.0 (unchanged) |
| Min spacing enforcement | None | 300mm gap |
| Simulation duration | 15 min | 15 min |

## 3. Summary Statistics

In [ ]:
def compute_summary(df_metrics, df_events, label):
    """Compute summary statistics from a simulation run."""
    if df_metrics is None:
        print(f'No data for {label}')
        return None
    
    # Get final row for cumulative metrics
    final = df_metrics.iloc[-1]
    duration_sec = df_metrics['sim_time_sec'].max()
    duration_min = duration_sec / 60.0
    
    # Count jams from events
    jam_count = 0
    if df_events is not None:
        jam_count = len(df_events[df_events['event_type'] == 'JAM'])
    
    # Count processed boxes from events
    box_exits = 0
    if df_events is not None:
        box_exits = len(df_events[df_events['event_type'].isin(['BOX_EXIT_B', 'BOX_EXIT_C'])])
    
    # Fault time (estimate from state = 3 rows)
    fault_rows = df_metrics[df_metrics['system_state'] == 3]
    fault_time_sec = len(fault_rows)  # Each row ~1 second of logging interval
    
    summary = {
        'Label': label,
        'Duration (min)': f'{duration_min:.1f}',
        'Boxes Processed': box_exits,
        'Avg Cycle Time (s)': f"{final.get('avg_cycle_time_sec', 0):.2f}",
        'Throughput (boxes/hr)': f"{final.get('throughput_per_hour', 0):.1f}",
        'Jam Events': jam_count,
        'Jams/Hour': f'{jam_count / (duration_min / 60.0):.1f}' if duration_min > 0 else '0',
        'Est. Fault Time (s)': fault_time_sec,
        'Est. Uptime (%)': f'{(1 - fault_time_sec / duration_sec) * 100:.1f}' if duration_sec > 0 else '0',
    }
    return summary

# Compute summaries
summary_baseline = compute_summary(df_baseline, ev_baseline, 'Baseline')
summary_improved = compute_summary(df_improved, ev_improved, 'Improved')

# Display comparison table
if summary_baseline and summary_improved:
    comparison = pd.DataFrame([summary_baseline, summary_improved]).set_index('Label').T
    display(comparison)
else:
    print('Run baseline and improved simulations first to generate data.')
    print('  Baseline: python process_sim.py --output-dir ../data/baseline')
    print('  Improved: python process_sim.py --output-dir ../data/improved')

## 4. Throughput Over Time

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

if df_baseline is not None:
    ax.plot(df_baseline['sim_time_sec'] / 60, df_baseline['throughput_per_hour'],
            label='Baseline', color='#E74C3C', alpha=0.8, linewidth=1.5)

if df_improved is not None:
    ax.plot(df_improved['sim_time_sec'] / 60, df_improved['throughput_per_hour'],
            label='Improved', color='#2ECC71', alpha=0.8, linewidth=1.5)

ax.axhline(y=60, color='#3498DB', linestyle='--', alpha=0.7, label='Target (60/hr)')

ax.set_xlabel('Time (minutes)')
ax.set_ylabel('Throughput (boxes/hour)')
ax.set_title('Throughput Over Time: Baseline vs. Improved')
ax.legend()
ax.set_ylim(bottom=0)
plt.tight_layout()
plt.savefig('throughput_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Box Count Over Time

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

if df_baseline is not None:
    ax.plot(df_baseline['sim_time_sec'] / 60, df_baseline['box_count'],
            label='Baseline', color='#E74C3C', alpha=0.8, linewidth=2)

if df_improved is not None:
    ax.plot(df_improved['sim_time_sec'] / 60, df_improved['box_count'],
            label='Improved', color='#2ECC71', alpha=0.8, linewidth=2)

ax.set_xlabel('Time (minutes)')
ax.set_ylabel('Cumulative Boxes Processed')
ax.set_title('Cumulative Box Count: Baseline vs. Improved')
ax.legend()
plt.tight_layout()
plt.savefig('box_count_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Jam Events Timeline

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

if ev_baseline is not None:
    jams_b = ev_baseline[ev_baseline['event_type'] == 'JAM']
    ax1.scatter(jams_b['sim_time_sec'] / 60, [1] * len(jams_b),
                marker='x', color='#E74C3C', s=100, label=f'Jams ({len(jams_b)})')
    ax1.set_ylabel('Baseline')
    ax1.set_title('Jam Events Over Time')
    ax1.legend()
    ax1.set_yticks([])

if ev_improved is not None:
    jams_i = ev_improved[ev_improved['event_type'] == 'JAM']
    ax2.scatter(jams_i['sim_time_sec'] / 60, [1] * len(jams_i),
                marker='x', color='#2ECC71', s=100, label=f'Jams ({len(jams_i)})')
    ax2.set_ylabel('Improved')
    ax2.legend()
    ax2.set_yticks([])

ax2.set_xlabel('Time (minutes)')
plt.tight_layout()
plt.savefig('jam_events_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. KPI Comparison Bar Chart

In [ ]:
if summary_baseline and summary_improved:
    kpis = ['Boxes Processed', 'Jam Events']
    baseline_vals = [int(summary_baseline[k]) for k in kpis]
    improved_vals = [int(summary_improved[k]) for k in kpis]

    x = np.arange(len(kpis))
    width = 0.35

    fig, ax = plt.subplots(figsize=(8, 5))
    bars1 = ax.bar(x - width/2, baseline_vals, width, label='Baseline',
                   color='#E74C3C', alpha=0.8)
    bars2 = ax.bar(x + width/2, improved_vals, width, label='Improved',
                   color='#2ECC71', alpha=0.8)

    ax.set_ylabel('Count')
    ax.set_title('Key Performance Indicators: Before vs. After')
    ax.set_xticks(x)
    ax.set_xticklabels(kpis)
    ax.legend()

    # Add value labels on bars
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{int(height)}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3), textcoords='offset points',
                        ha='center', va='bottom', fontweight='bold')

    plt.tight_layout()
    plt.savefig('kpi_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()

## 8. Cycle Time Distribution

In [ ]:
def extract_cycle_times(ev_df):
    """Extract cycle times from events DataFrame."""
    if ev_df is None:
        return []
    exits = ev_df[ev_df['event_type'].isin(['BOX_EXIT_B', 'BOX_EXIT_C'])]
    cycle_times = []
    for _, row in exits.iterrows():
        desc = row['description']
        if 'cycle=' in desc:
            ct_str = desc.split('cycle=')[1].replace('s', '')
            try:
                cycle_times.append(float(ct_str))
            except ValueError:
                pass
    return cycle_times

ct_baseline = extract_cycle_times(ev_baseline)
ct_improved = extract_cycle_times(ev_improved)

if ct_baseline or ct_improved:
    fig, ax = plt.subplots(figsize=(10, 5))
    
    if ct_baseline:
        ax.hist(ct_baseline, bins=20, alpha=0.6, color='#E74C3C',
                label=f'Baseline (avg={np.mean(ct_baseline):.2f}s)', edgecolor='white')
    if ct_improved:
        ax.hist(ct_improved, bins=20, alpha=0.6, color='#2ECC71',
                label=f'Improved (avg={np.mean(ct_improved):.2f}s)', edgecolor='white')
    
    ax.set_xlabel('Cycle Time (seconds)')
    ax.set_ylabel('Frequency')
    ax.set_title('Cycle Time Distribution: Baseline vs. Improved')
    ax.legend()
    plt.tight_layout()
    plt.savefig('cycle_time_distribution.png', dpi=150, bbox_inches='tight')
    plt.show()

## 9. Conclusions and Discussion

### Results Summary

After running both scenarios with identical input conditions (72 boxes/hr arrival
rate, same conveyor speed), the improved configuration demonstrates:

1. **Reduced false jam detections** -- The increased timeout (4.0s -> 5.5s) allows
   boxes that are momentarily delayed at photoeyes to clear without triggering
   a fault, reducing unnecessary downtime.

2. **Maintained or improved throughput** -- Despite the longer timeout, throughput
   remains at or above the 60 boxes/hr target because the system spends less time
   in FAULT state.

3. **Higher uptime percentage** -- Fewer jam events translates directly to less
   fault recovery downtime.

### Trade-offs

- A longer jam timeout means a real jam (e.g., physical obstruction) takes longer
  to detect. In a real system, this trade-off must be balanced against the cost
  of false positives.
- Adding inter-arrival spacing enforcement slightly reduces the maximum
  theoretical throughput but prevents box-on-box collisions that cause jams.

### Applicability to Real MHE Systems

This same data-driven approach applies to real fulfillment center conveyors:

- **Monitor** KPIs (throughput, jam rate, downtime) using SCADA/historian data.
- **Hypothesize** parameter or logic changes based on observed patterns.
- **Test** changes in a controlled manner (off-shift, single lane, etc.).
- **Measure** the impact and iterate.

Key areas for continuous improvement in real systems:
- Photoeye sensitivity and placement
- Conveyor speed profiles at merge/divert points
- Inter-package gap control
- Predictive maintenance triggers based on jam frequency trends